# How to use mlflow client

In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
client.search_experiments()

[<Experiment: artifact_location='file:///d:/github/mlops-zoomcamp-course/module-02/mlruns/1', creation_time=1716716457026, experiment_id='1', last_update_time=1716716457026, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716715813570, experiment_id='0', last_update_time=1716715813570, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
#make new experiment

client.create_experiment('my-cool-experiment')

'2'

In [6]:
#find the best runs
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1', #nyc-taxi-experiment
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY, #only active model not deleted
    max_results=5,
    order_by=['metrics.rmse ASC']
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ddb9659bd80a464c8eade76b1a60450d, rmse: 6.3040
run id: 5bf77dc70a50439cb62ecbf5d62113e0, rmse: 6.3040
run id: 26e1650b7e574d529b00242d7920c44a, rmse: 6.3040
run id: 302ab20875224b51bc26bc0d3513c047, rmse: 6.3045
run id: ec31144ac6c74c7ca9eba9fbc8edafd6, rmse: 6.3073


In [8]:
#find the best runs and use filters
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1', #nyc-taxi-experiment
    filter_string='metrics.rmse > 6.4',
    run_view_type=ViewType.ACTIVE_ONLY, #only active model not deleted
    max_results=5,
    order_by=['metrics.rmse ASC']
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: b79d3e8fbd2545e6a0eca69c8b9b9ba1, rmse: 6.4006
run id: c67576c9fb4b47b28ade3f2f837c1c62, rmse: 6.4016
run id: e7e9296b4c484f46abe39df979d838c2, rmse: 6.4083
run id: 5119638776e443f686b9d9ee8ae6a772, rmse: 6.4094
run id: 0fbb6f592bc04c51b8aa493eca051169, rmse: 6.4130


## Promote model registry

In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = 'b79d3e8fbd2545e6a0eca69c8b9b9ba1'
model_uri = f'runs/{run_id}/model'
mlflow.register_model(model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716820695631, current_stage='None', description=None, last_updated_timestamp=1716820695631, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/b79d3e8fbd2545e6a0eca69c8b9b9ba1/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

## How to transition model from one stage to another

In [11]:
model_name = 'nyc-taxi-regressor'
latest_version = client.get_latest_versions(name=model_name)
latest_version


C:\Users\asus\AppData\Local\Temp\ipykernel_8228\633429021.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1716820695631, current_stage='None', description=None, last_updated_timestamp=1716820695631, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/b79d3e8fbd2545e6a0eca69c8b9b9ba1/model', status='READY', status_message=None, tags={}, user_id=None, version=2>]

In [12]:
for version in latest_version:
    print(f'version: {version.version}, stage: {version.current_stage}')

version: 2, stage: None


## How to transit model stage

stage removed so will use alias/tags

In [15]:
model_version = '2'
new_stage = 'Staging'
client.set_registered_model_alias(
    name=model_name,
    alias='Staging',
    version='2'
)

In [16]:
from datetime import datetime
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f'The model version {model_version} was transitioned to {new_stage} on {date}'
)

<ModelVersion: aliases=['Staging'], creation_timestamp=1716820695631, current_stage='None', description='The model version 2 was transitioned to Staging on 2024-05-27', last_updated_timestamp=1716823523788, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/b79d3e8fbd2545e6a0eca69c8b9b9ba1/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

# Promot and archive models

In [39]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, X_test, y_test):
    model = mlflow.pyfunc.load_model(f'runs:/{name}/models_mlflow')
    y_pred= model.predict(X_test)
    return {'rmse': mean_squared_error(y_test, y_pred, squared=False)}

In [21]:
df = read_dataframe('./data/green_tripdata_2021-03.parquet')

In [23]:
#yang ada processornya 
run_id = 'ddb9659bd80a464c8eade76b1a60450d'

In [24]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'd:\\github\\mlops-zoomcamp-course\\module-02\\preprocessor'

In [26]:
import pickle 

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [29]:
X_test = preprocess(df, dv)

In [30]:
target = 'duration'
y_test = df[target].values

In [40]:
%time test_model(name=run_id, X_test=X_test, y_test=y_test)

c:\Users\asus\.virtualenvs\module-02-jXzagZlw\lib\site-packages\xgboost\core.py:160: UserWarning: [23:34:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 1min 9s
Wall time: 15.3 s


c:\Users\asus\.virtualenvs\module-02-jXzagZlw\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.253233230603499}